In [1]:
def s(x): return " "*(10+x)
print(s(5)+".\n"+s(4)+"..:\n"+s(2)+"Hultnér\n"+s(0)+"Technologies\n\n@ahultner | https://hultner.se/")

               .
              ..:
            Hultnér
          Technologies

@ahultner | https://hultner.se/


# Intro to pydantic
### Run-Time Type Checking For Your Dataclasses

## Index
- Quick refresher on python data classes
- Pydantic introduction
    - Prior art
    - Minimal example from dataclass
    - Runtime type-checking
    - JSON (de)serialisation
    - JSONSchema
    - Validators
        - [Custom model validators](https://pydantic-docs.helpmanual.io/usage/validators/)
        - [Validation decorator for functions](https://pydantic-docs.helpmanual.io/usage/validation_decorator/), via `@validate_arguments`. Still in beta, API may change.
    - FastAPI framework
        - OpenAPI Specifications
        - Autogenerated tests
- Cool features worth mentioning
- Future
- Conclusion
    

Let's start with a quick `@dataclass`-refresher.

I love waffles, don't everyone?
For our examples we'll use our imaginary café, "The Waffle Bistro" 🧇 🌟

In [164]:
from dataclasses import dataclass
from typing import Tuple

In [165]:
@dataclass
class Waffle:
    style: str
    toppings: Tuple[str, ...]
    

In [166]:
Waffle("Swedish", ("chocolate sauce", "ham"))

Waffle(style='Swedish', toppings=('chocolate sauce', 'ham'))

Now we may want to constrain the toppings to ones we actually offer
🥛 🍓 🟠  🍫

We offer a couple of cream based toppings, and a couple of _"dessert sauces"_

In [167]:
from typing import Union
from enum import Enum

In [168]:
class Cream(str, Enum):
    whipped_cream = "whipped cream"
    ice_cream = "icecream"

class DessertSauce(str, Enum):
    cloudberry_jam = "cloudberry jam"
    raspberry_jam = "raspberry jam"
    choclate_sauce = "chocolate sauce"

Topping = Union[DessertSauce, Cream]

class WaffleStyle(str, Enum):
    swedish = "Swedish"
    belgian = "Belgian"


@dataclass
class Waffle:
    style: WaffleStyle
    toppings: Tuple[Topping, ...]

Let's see what happens if we try to create a waffle with ham  topping.

In [100]:
Waffle("Swedish", ("chocolate sauce", "ham"))

Waffle(style='Swedish', toppings=('chocolate sauce', 'ham'))

With dataclasses the types aren't enforced, this can ofcourse be implemented but in this case we'll lean on the shoulders of a giant, pydantic 🧹🐍🧐

In [101]:
from pydantic.dataclasses import dataclass

@dataclass
class Waffle:
    style: WaffleStyle
    toppings: Tuple[Topping, ...]

As you can see the only thing changed in this example is that we import the dataclass decorator from pydantic.

In [102]:
from pydantic import ValidationError
try:
    Waffle("Swedish", ("chocolate sauce", "ham"))
except ValidationError as err:
    print(err)

2 validation errors for Waffle
toppings -> 1
  value is not a valid enumeration member; permitted: 'cloudberry jam', 'raspberry jam', 'chocolate sauce' (type=type_error.enum; enum_values=[<DessertSauce.cloudberry_jam: 'cloudberry jam'>, <DessertSauce.raspberry_jam: 'raspberry jam'>, <DessertSauce.choclate_sauce: 'chocolate sauce'>])
toppings -> 1
  value is not a valid enumeration member; permitted: 'whipped cream', 'icecream' (type=type_error.enum; enum_values=[<Cream.whipped_cream: 'whipped cream'>, <Cream.ice_cream: 'icecream'>])


With that simple change we can see that our new instance of an unsupported waffle actually raises errors 🚫🚨

These errors are very readable!

So let's try to create a valid waffle ✅

In [103]:
Waffle("Swedish", (Cream.whipped_cream, "cloudberry jam"))

Waffle(style=<WaffleStyle.swedish: 'Swedish'>, toppings=(<Cream.whipped_cream: 'whipped cream'>, <DessertSauce.cloudberry_jam: 'cloudberry jam'>))

See how the cloudberry jam was automatically parsed as a Dessert Sauce. This is a feature of pydantic, more strictness can be achieved with the strict types and a fully [strict mode](https://github.com/samuelcolvin/pydantic/issues/1098) is being worked on.

So what about JSON? 🧑‍💻  
The dataclass dropin replacement decorator from pydantic is great for compability but by using `pydantic.BaseModel` we can get even more out of pydantic. One of those things is (de)serialisation, pydantic have native support JSON encoding and decoding.

In [169]:
from pydantic import BaseModel

class Waffle(BaseModel):
    style: WaffleStyle
    toppings: Tuple[Topping, ...]

*Disclaimer: Pydantic is primarly a parsing library and does validation as a means to an end, so make sure it makes sense for you.*

When using the BaseModel the default behaviour requires to specify the init arguments using their keywords like below

In [105]:
Waffle(style="Swedish", toppings=(Cream.whipped_cream, "cloudberry jam"))

Waffle(style=<WaffleStyle.swedish: 'Swedish'>, toppings=(<Cream.whipped_cream: 'whipped cream'>, <DessertSauce.cloudberry_jam: 'cloudberry jam'>))

We can now easily encode this object as `JSON`, there's also [built-in support](https://pydantic-docs.helpmanual.io/usage/exporting_models/) for dict, pickle, immutable `copy()`. Pydantic will also (de)serialise subclasses.

In [106]:
_.json()

'{"style": "Swedish", "toppings": ["whipped cream", "cloudberry jam"]}'

And we can also reconstruct our original object using the `parse_raw`-method.

In [107]:
Waffle.parse_raw('{"style": "Swedish", "toppings": ["whipped cream", "cloudberry jam"]}')

Waffle(style=<WaffleStyle.swedish: 'Swedish'>, toppings=(<Cream.whipped_cream: 'whipped cream'>, <DessertSauce.cloudberry_jam: 'cloudberry jam'>))

Errors raises a validation error, these can also be represented as JSON.

In [124]:
try:
    Waffle(style=42, toppings=(Cream.whipped_cream, "cloudberry jam"))
except ValidationError as err:
    print(err.json())

[
  {
    "loc": [
      "style"
    ],
    "msg": "value is not a valid enumeration member; permitted: 'Swedish', 'Belgian'",
    "type": "type_error.enum",
    "ctx": {
      "enum_values": [
        "Swedish",
        "Belgian"
      ]
    }
  }
]


We can also export a JSONSchema directly from our model, this is very useful for instance if we want to use your model to feed a Swagger/OpenAPI-spec. 📜✅

⚠ *Caution: Pydantic uses draft 7 of JSONSchema, this is used in the just released OpenAPI 3.1 spec.  
The still common 3.0.x spec uses draft 4.  
I spoke with Samuel Colvin, the creator of pydantic about this and his recommendation is to write a `schema_extra`function to use the older JSONSchema version if you want strict compability. The FastAPI framework doesn't do this and is slightly incompatible with the older OpenAPI-spec*

In [125]:
Waffle.schema()

{'title': 'Waffle',
 'type': 'object',
 'properties': {'style': {'$ref': '#/definitions/WaffleStyle'},
  'toppings': {'title': 'Toppings',
   'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/DessertSauce'},
     {'$ref': '#/definitions/Cream'}]}}},
 'required': ['style', 'toppings'],
 'definitions': {'WaffleStyle': {'title': 'WaffleStyle',
   'description': 'An enumeration.',
   'enum': ['Swedish', 'Belgian'],
   'type': 'string'},
  'DessertSauce': {'title': 'DessertSauce',
   'description': 'An enumeration.',
   'enum': ['cloudberry jam', 'raspberry jam', 'chocolate sauce'],
   'type': 'string'},
  'Cream': {'title': 'Cream',
   'description': 'An enumeration.',
   'enum': ['whipped cream', 'icecream'],
   'type': 'string'}}}

That was the basics using the built-in validators, but what if you want to implement your own business rules in a custom validator, we're going to look at this next.

We now want to add some custom busniess logic specific for "The Waffle Bistro".
In this case we want to only allow:
- Either icecream or whipped cream 🍦 ⊕🥛
- Jam for Swedish waffles 🇸🇪 🟠 🔴
- Choclate for Belgian waffles 🇧🇪 🍫

In [172]:
from pydantic import validator, root_validator

swedish_toppings = (
    DessertSauce.raspberry_jam, DessertSauce.cloudberry_jam,
)

belgian_toppings = (DessertSauce.choclate_sauce,)

class WaffleOrder(Waffle):

    
    # Root validators check the entire model
    @root_validator(pre=False)
    def check_style_topping(cls, values):
        style, toppings = values.get("style"), values.get("toppings")
        # Check swedish style
        if (
            style == WaffleStyle.swedish and 
            all(t in swedish_toppings for t in toppings if type(t) is DessertSauce)
        ):
            return values
        
        # Check belgian style
        if (
            style == WaffleStyle.belgian and 
            all(t in belgian_toppings for t in toppings if type(t) is DessertSauce)
        ):
            return values
      
        # Doesn't match any of our allowed styles
        raise ValueError(f"The Waffle Bistro doesn't sell this waffle.")
    
        
    # A validator looking at a single property
    @validator('toppings')
    def check_cream(cls, toppings):
        creams = [t for t in toppings if type(t) is Cream]
        if len(creams) > 1:
            raise ValueError(f"We only allow for one cream topping, given: {creams}")
        return toppings


Now let's see if we create some invalid waffles 🧇 ⚠️🚨

In [173]:
try: 
    WaffleOrder(style="Swedish", toppings=["icecream", "whipped cream", "cloudberry jam"])
except ValidationError as err:
    print(err)

2 validation errors for WaffleOrder
toppings
  We only allow for one cream topping, given: [<Cream.ice_cream: 'icecream'>, <Cream.whipped_cream: 'whipped cream'>] (type=value_error)
__root__
  'NoneType' object is not iterable (type=type_error)


In [174]:
try: 
    WaffleOrder(style="Swedish", toppings=["icecream", "cloudberry jam", "chocolate sauce"])
except ValidationError as err:
    print(err)

1 validation error for WaffleOrder
__root__
  The Waffle Bistro doesn't sell this waffle. (type=value_error)


Now let's create a waffle 🧇 allowed by our rules! ✨

In [156]:
WaffleOrder(style="Belgian", toppings=["whipped cream", "chocolate sauce"])

WaffleOrder(style=<WaffleStyle.belgian: 'Belgian'>, toppings=(<Cream.whipped_cream: 'whipped cream'>, <DessertSauce.choclate_sauce: 'chocolate sauce'>))

Gosh these runtime type checkers are rather useful, but what about **functions**? 

Pydantic got you covered with `@validate_arguments`. *Still in beta, API may change, release 2020-04-18 in version 1.5*

In [175]:
from pydantic import validate_arguments

# Validator on function
# Ensure valid waffles when making orders
@validate_arguments
def make_order(waffle: WaffleOrder):
    ...
    

In [161]:
try:
    make_order({
        "style":"Breakfast",
        "toppings":("whipped cream", "raspberry jam")
    })
except ValidationError as err:
    print(err)

2 validation errors for MakeOrder
waffle -> style
  value is not a valid enumeration member; permitted: 'Swedish', 'Belgian' (type=type_error.enum; enum_values=[<WaffleStyle.swedish: 'Swedish'>, <WaffleStyle.belgian: 'Belgian'>])
waffle -> __root__
  The Waffle Bistro doesn't sell this waffle. (type=value_error)


## FastAPI
FastAPI is a lean microframework similar to Flask which utilizes pydantic models heavily, it will also automatically generate OpenAPI-specifications from your application based on your models.

This gives you framework agnostic models while still being able to leverage tight integration with a modern and easy to use framework. If you're going to start a new API-project i highly recommend trying FastAPI.

In [176]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

def make_order(waffle: WaffleOrder):
    # Business logic for making an order
    pass

def dispatch_order(waffle: WaffleOrder):
    # Hand over waffle customer
    pass

# Deliver a waffle
@app.post("/delivery/waffle")
async def deliver_waffle_order(waffle: WaffleOrder):
    dispatch = dispatch_order(waffle)
    return dispatch

@app.post("/order/waffle")
async def order_waffle(waffle: WaffleOrder):
    order = make_order(waffle)
    return order


This is everything we need to create a small API around our models.

---

That's it, a quick introduction to pydantic! 

But this is just the tip of the iceberg 🗻 and I want to give you a hint about what more can be done.  
I'm not going to go into detail in any of this but feel free to ask me about it in the chat, on Twitter/LinkedIn or via email. 💬📨

## Cool features worth mentioning

- Post **1.0**, reached this milestone about a year ago
- Support for [standard library types](https://pydantic-docs.helpmanual.io/usage/types/#pydantic-types)
- Offer useful extra types for every day use
    - Email
    - HttpUrl (and more, stricturl for custom validation)
    - PostgresDsn
    - IPvAnyAddress (as well as IPv4Address and IPv6Address from ipaddress)
    - PositiveInt
    - PaymentCardNumber, PaymentCardBrand.[amex, mastercard, visa, other], checks luhn, str of digits and BIN-based lenght.
    - [Constrained types](https://pydantic-docs.helpmanual.io/usage/types/#constrained-types) (e.g. conlist, conint, etc.)
    - and more…
- Supports [custom datatypes](https://pydantic-docs.helpmanual.io/usage/types/#custom-data-types)
- [Settings management](https://pydantic-docs.helpmanual.io/usage/settings/)
    - Typed configuration management
    - Automatically reads from environment variables
    - Dotenv (`.env`) support via defacto standard [python-dotenv](https://pypi.org/project/python-dotenv/).
- ORM-mode
- Recursive models
- Works with mypy out of the box, [mypy plugin](https://pydantic-docs.helpmanual.io/mypy_plugin/) further improves experience.
- [Postponed annotations, self-referencing models](https://pydantic-docs.helpmanual.io/usage/postponed_annotations/), [PEP-563](https://www.python.org/dev/peps/pep-0563/)-style.
- python-devtools intergration
- PyCharm plugin
- [Fast](https://pydantic-docs.helpmanual.io/benchmarks/) compared to popular alternatives!  
  But always make your own benchmarks for your own usecase if performance is important for you.

## Future
- A strict mode is being worked on, in the future this will enable us to choose between Strict and Coercion on a model level instead of relying on the Strict* types.
- The project is very active and a lot of improvements are constantly being made to the library.
    

## Conclusion
Pure python syntax
Better validation
Very useful JSON-tools for API's
Easy to migrate from dataclasses
Lots of useful features
Try it out!

## Want to hear more from me? 
I'm making a course on property based testing in python using Hypothesis.  
[Sign up here](https://forms.gle/yRWapypPXdPFSLME7)